In [ ]:
import sys;
sys.path.insert(0, '..')

In [ ]:
from math import pi, sqrt

from util import cis, all_close
from sim_circuit import *

## Exercise 1

Using the angle parameter $\theta = 12.85$ and $n = 4$ qubits, use both methods (integer and ratio of magnitudes) to get an estimate for $\theta$ using 100 shots.

**Answer:**

In [ ]:
def encode_frequency(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])

    for j in range(n):
        qc.p(2 * pi / 2 ** (n - j) * v, q[j])

    qc.report('geometric_sequence')

    qc.iqft(range(n))

    qc.report('iqft')

    return qc

We can get the value $v$ to encode using $v = \theta \frac{2^n}{2 \pi}$:

$v = 12.85 \frac{2^4}{2 \pi}$

In [ ]:
n = 4
theta = 12.85 * (pi/180) #convert to radians
v = theta*2**n/(2*pi)
v

In [ ]:
n = 4

qc = encode_frequency(n, v)

In [ ]:
result = qc.measure(shots = 100)

In [ ]:
result['counts']

The most frequent outcome is 1, so the estimate using the most frequent outcome method is

$\theta = 1 \frac{2 \pi}{2^n}$

In [ ]:
1*2*pi/(2**n)

The estimate using the two most frequent outcomes is (note, in this case the lower of the two most frequent outcomes is 0 so we estimate the integer part by 0)

In [ ]:
p_0 = result['counts'][0]/sum(result['counts'].values())
p_1 = result['counts'][1]/sum(result['counts'].values())

decimal_estimate = sqrt(p_1)/(sqrt(p_0)+ sqrt(p_1))

decimal_estimate

## Exercise 2

We can describe the eigenstate in this example as

$$\vert \psi \rangle = \begin{bmatrix}\frac{i}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{bmatrix}$$

The operator is a latexmath:[$R_Y$]-gate with angle parameter latexmath:[$2\theta$].
The two-by-two matrix form of an latexmath:[$R_Y$]-gate is

$$\begin{bmatrix} \cos \theta & -\sin \theta\\ \sin \theta & \cos \theta \end{bmatrix}$$

We can express the application of the operator to the eigenstate as

$$R_Y(2\theta) \vert \psi \rangle$$

Show that the complex number $\text{cis} \left( \theta \right) $ is the eigenvalue of the eigenstate $\vert \psi \rangle$.


**Answer:**

We can express the application of the operator to the eigenstate as

$$\begin{align}
R_Y(2\theta) \vert \psi \rangle & = \begin{bmatrix} \cos \theta & -\sin \theta\\ \sin \theta & \cos \theta \end{bmatrix} \begin{bmatrix}\frac{i}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{bmatrix} \\
& = \begin{bmatrix}\frac{i}{\sqrt{2}}\cos \theta - \frac{1}{\sqrt{2}}\sin \theta\\ \frac{i}{\sqrt{2}}\sin \theta + \frac{1}{\sqrt{2}}\cos \theta \end{bmatrix} \\
& = \begin{bmatrix}\frac{i}{\sqrt{2}}\left( \cos \theta + i \sin \theta \right) \\ \frac{1}{\sqrt{2}} \left( i\sin \theta + cos \theta \right) \end{bmatrix} \\
& = \begin{bmatrix}\frac{i}{\sqrt{2}} \text{cis} \left( \theta \right)  \\ \frac{1}{\sqrt{2}} \text{cis} \left( \theta \right) \end{bmatrix} = \text{cis} \left( \theta \right) \vert \psi \rangle
\end{align}
$$

The complex number $\text{cis} \left( \theta \right) $ is the eigenvalue of the eigenstate $\vert \psi \rangle$.


## Exercise 3

Let's look at the Phase gate in the context of eigenstates and eigenvalues.
An eigenstate of the circuit can be created by applying an X-gate to a single qubit in its initial state.
The Python code below creates this state:

In [ ]:
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.x(q[0])

state = qc.run()

Apply the circuit (a phase gate with angle parameter $\theta$) to the state created above using various values for $\theta$.
Check that each time the circuit is applied, the amplitudes of the state are multiplied by $\text{cis}(\theta)$.

**Answer:**

In [ ]:
print_state_table(state)

In [ ]:
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.x(q[0])

theta = pi
qc.p(theta, q[0])

state1 = qc.run()
print_state_table(state1)

In [ ]:
assert all_close(state1, [a*cis(theta) for a in state])

In [ ]:
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.x(q[0])

theta = 3*pi/2
qc.p(theta, q[0])

state2 = qc.run()
print_state_table(state2)

In [ ]:
assert all_close(state2, [a*cis(theta) for a in state])

In [ ]:
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.x(q[0])

theta = pi/2
qc.p(theta, q[0])

state3 = qc.run()
print_state_table(state3)

In [ ]:
assert all_close(state3, [a*cis(theta) for a in state])

## Exercise 4

Use the state created in figure 9.22 to get an estimate for the true angle parameter $\theta$.

**Answer:**

In [ ]:
n = 3
q = QuantumRegister(n)
a = QuantumRegister(1)
qc = QuantumCircuit(q, a)

qc.x(a[0])
qc.rx(-pi/2, a[0])

for i in range(n):
    qc.h(q[i])

theta = 4.7*2*pi/2**n

for i in range(n):
    for _ in range(2**i):
        qc.cry(2*theta, q[i], a[0])

qc.rx(pi/2, a[0])
qc.x(a[0])

qc.append_iqft(q)

In [ ]:
result = qc.measure(shots = 100)

In [ ]:
print_state_table(result['state vector'])

In [ ]:
result['counts']

We can estimate the integer part by 4.

In [ ]:
p_4 = result['counts'][4]/sum(result['counts'].values())
p_5 = result['counts'][5]/sum(result['counts'].values())

decimal_estimate = sqrt(p_5)/(sqrt(p_4)+ sqrt(p_5))

v_estimate = 4 + decimal_estimate

The estimate for $v$ is

In [ ]:
v_estimate

Estimate for $\theta$ (in radians):

In [ ]:
v_estimate*2*pi/(2**n)